# Importação de pacotes

In [1]:
import pandas as pd
import numpy as np
import re
from itertools import groupby
from os import listdir, path
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import calendar

# Proveniência

In [ ]:
#Import ProvDocument to you Python Code
from prov.model import ProvDocument

# Create a new provenance document
doc1 = ProvDocument()  # d1 is now an empty provenance document
# Declaring namespaces for various prefixes used in the example
doc1.add_namespace('now', 'http://www.provbook.org/nownews/')
doc1.add_namespace('nowpeople', 'http://www.provbook.org/nownews/people/')
doc1.add_namespace('bk', 'http://www.provbook.org/ns/#')

# Adding an Entity: now:employment-article-v1.html
e1 = doc1.entity('now:employment-article-v1.html')
# Adding an Agent: nowpeople:Bob
doc1.agent('nowpeople:Anderson')

# Attributing the article to the agent
doc1.wasAttributedTo(e1, 'nowpeople:Anderson')

# Printing what we have so far (serialized in PROV-N)
print(doc1.get_provn())




In [8]:
#definicões de variáveis
path_inmet = 'datasets/inmet' #caminho para os arquivos do inmet
path_output = 'datasets/agregados' #caminho para onde deseja que as planilhas por estação sejam inseridas
padronizar_dados = True #defina true se deseja que a padronização seja feita (precisa dos arquivos do inmet)

In [9]:
## Conjunto de funções para unificar e padronizar os datasets
colunas_importantes = [0, 1, 2, 6, 9, 10, 13, 14, 18]
index_metadados = ['REGIÃO', 'UF', 'ESTAÇÃO', 'CODIGO (WMO)', 'LATITUDE', 'LONGITUDE', 'ALTITUDE', 'DATA DE FUNDAÇÃO']

 #troca / por - e adiciona o 20 na frente
def formata_data(dt):
    if '/' not in dt: return dt
    d, m, a = dt.split('/')
    return f'20{a}-{m}-{d}'


#troca / por - e remove 'UTC' das strings de hora
def formata_data_hora(mi):
    data, hora = mi[0].replace('/', '-'), mi[1].replace(' UTC', '')
    if len(hora) == 4:  hora = hora[:2] + ':' + hora[2:]
    return data, hora


#faz a leitura de cada arquivo e concatena
def concat_years(code, file_list, output_dir):
    lla_data = [] #geographic data
    sensor_data = [] #sensor data
    for file in file_list:
        #leitura dos sensores
        df = pd.read_csv(file, skiprows=8, encoding='latin_1', sep=';', decimal=',', usecols=colunas_importantes, index_col=[0, 1], na_values=[-9999])
        df = df.rename_axis(['Data', 'Hora']).rename(columns={ df.columns[1]: 'RADIACAO GLOBAL (KJ/m²)'})
        sensor_data.append(df)
        
        #leitura dos dados geograficos
        md = pd.read_csv(file, encoding='latin_1', sep=';', decimal=',', skiprows=4, nrows=3, header=None, usecols=[1], na_values=['F'], names=[df.index[0][0][:4]])
        lla_data.append(md)
    
    #concatena os dados geograficos
    md = pd.concat(lla_data, axis=1, copy=False)
    md.index = ['LATITUDE', 'LONGITUDE', 'ALTITUDE']
    
    #concatena, arruma os index e escreve os dados para um arquivo
    df = pd.concat(sensor_data, copy=False).replace(-9999, np.nan)
    df.index = df.index.map(formata_data_hora)
    
    #salva todas as leituras em um arquivo por sensor
    last = '_'.join(file_list[-1].split('_')[1:5])
    path = f'{output_dir}/{last}.csv'
    md.to_csv(path, sep=';')
    df.sort_index().to_csv(path, sep=';', mode='a')

    
def unify_data(inmet_dir, output_dir):
    #salvando o nome de todas as planilhas
    arquivos = []
    for folder in listdir(inmet_dir):
        c = f'{inmet_dir}/{folder}'
        if not path.isdir(c): continue
        if path.isdir(f'{c}/{folder}'): c = f'{c}/{folder}'
        arquivos += [f'{c}/{a}' for a in listdir(c) if a.endswith('.CSV')]
    
    #para cada codigo, concatena os anos e salva os metadados
    metadatas = []
    search_groups = lambda s: re.search('_([A-Z][0-9]{3})_', s).group(1)
    for k, grupo in groupby(sorted(arquivos, key=search_groups), search_groups):
        concat_years(k, sorted(grupo), output_dir)
        print(f'{k} OK')

In [10]:
 #retorna maximos e mínimos para cada coluna em todos os arquivos
def checking_bounds(path):
    tabelas = {
        'min': [], #minimo de cada coluna
        'max': [], #maximo de cada coluna
    }
      
    for file in listdir(path):
        data = pd.read_csv(path+file, sep=';', index_col = [0, 1], skiprows=4)
        tabelas['min'].append(data.min().rename(file[:-4]))
        tabelas['max'].append(data.max().rename(file[:-4]))
    
    for k, v in tabelas.items():
        df = pd.DataFrame(v)
        df.columns = [f'{k.upper()} - {c}' for c in df.columns]
        tabelas[k] = df
        
    colunas = [v.columns for k, v in tabelas.items()]
    colunas = [j for i in zip(*colunas) for j in i]
    return pd.concat(tabelas.values(), axis=1).reindex(columns=colunas).sort_index()

###dados.apply(lambda x: x.groupby(x.notna().cumsum()).cumcount().max()).rename(local)

In [11]:
#retorna os dias marcados com True se estavam off ou False caso contrário
def days_off(file):
    data = pd.read_csv(file, sep=';', index_col = [0, 1], skiprows=4) \
                        .isna().apply(lambda row: all(row), axis = 1) \
                        .groupby(level=0).apply(lambda group: sum(group) == 24)
    data.name = file[:-4].split('/')[-1]
    return data


# retorna porcentagem de dias que a estação ficou off no ano
def percentage_off_per_year(path):
    stations = pd.concat([days_off(path+file) for file in listdir(path)], axis=1)
    return stations.groupby(lambda x: x.split('-')[0] ).apply(lambda x: x.sum(min_count=1)/ len(x) ).T.sort_index()

In [13]:
if padronizar_dados:
    unify_data(path_inmet, path_output)

A001 OK
A002 OK
A003 OK
A005 OK
A009 OK
A010 OK
A011 OK
A012 OK
A013 OK
A014 OK
A015 OK
A016 OK
A017 OK
A018 OK
A019 OK
A020 OK
A021 OK
A022 OK
A023 OK
A024 OK
A025 OK
A026 OK
A027 OK
A028 OK
A029 OK
A031 OK
A032 OK
A033 OK
A034 OK
A035 OK
A036 OK
A037 OK
A038 OK
A039 OK
A040 OK
A041 OK
A042 OK
A043 OK
A044 OK
A045 OK
A046 OK
A047 OK
A048 OK
A049 OK
A050 OK
A051 OK
A052 OK
A053 OK
A054 OK
A055 OK
A056 OK
A101 OK
A102 OK
A104 OK
A108 OK
A109 OK
A110 OK
A111 OK
A112 OK
A113 OK
A117 OK
A119 OK
A120 OK
A121 OK
A122 OK
A123 OK
A124 OK
A125 OK
A126 OK
A128 OK
A133 OK
A134 OK
A135 OK
A136 OK
A137 OK
A138 OK
A140 OK
A144 OK
A201 OK
A202 OK
A203 OK
A204 OK
A205 OK
A206 OK
A207 OK
A209 OK
A210 OK
A211 OK
A212 OK
A213 OK
A214 OK
A215 OK
A216 OK
A217 OK
A218 OK
A219 OK
A220 OK
A221 OK
A222 OK
A223 OK
A224 OK
A225 OK
A226 OK
A227 OK
A228 OK
A229 OK
A230 OK
A231 OK
A232 OK
A233 OK
A234 OK
A235 OK
A236 OK
A237 OK
A238 OK
A239 OK
A240 OK
A241 OK
A242 OK
A244 OK
A246 OK
A247 OK
A248 OK
A249 OK
A250 OK


In [14]:
bounds = checking_bounds('datasets/agregados/')
bounds.to_csv('datasets/extremos.csv', sep=';')
bounds

,"MIN - PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","MAX - PRECIPITAÇÃO TOTAL, HORÁRIO (mm)",MIN - RADIACAO GLOBAL (KJ/m²),MAX - RADIACAO GLOBAL (KJ/m²),MIN - TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),MAX - TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),MIN - TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),MAX - TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),MIN - UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),MAX - UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),MIN - UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),MAX - UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"MIN - VENTO, VELOCIDADE HORARIA (m/s)","MAX - VENTO, VELOCIDADE HORARIA (m/s)"
CO_DF_A001_BRASILIA,0.0,37.6,0.0,4205.3,9.4,36.5,8.5,34.4,12.0,97.0,10.0,97.0,0.1,7.1
CO_DF_A042_BRAZLANDIA,0.0,96.0,0.0,4168.7,11.6,35.8,10.5,34.3,12.0,97.0,10.0,96.0,0.0,9.7
CO_DF_A045_AGUAS EMENDADAS,0.0,42.4,0.0,4265.0,6.9,37.8,6.0,35.6,11.0,96.0,9.0,96.0,0.1,5.9
CO_DF_A046_GAMA (PONTE ALTA),0.0,41.2,0.0,4017.2,7.7,37.3,6.5,35.7,12.0,95.0,9.0,95.0,0.0,9.1
CO_DF_A047_PARANOA (COOPA-DF),0.0,70.2,0.0,4155.9,10.0,37.1,9.1,35.3,17.0,100.0,14.0,100.0,0.1,8.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S_SC_A867_ARARANGUA,0.0,42.4,0.0,4192.0,4.1,35.6,2.5,34.8,9.0,100.0,25.0,100.0,0.1,12.4
S_SC_A868_ITAJAI,0.0,32.6,0.0,4280.5,4.5,36.1,4.2,33.9,26.0,96.0,23.0,95.0,0.0,11.4
S_SC_A870_RANCHO QUEIMADO,0.0,38.4,0.0,4501.8,3.1,30.9,1.8,30.2,30.0,100.0,11.0,100.0,0.2,6.7
S_SC_A895_CHAPECO,0.0,38.0,0.0,4104.5,1.7,36.8,1.0,35.1,18.0,98.0,13.0,98.0,0.2,16.1


In [15]:
df = percentage_off_per_year('datasets/agregados/')
df.index = df.index.str.split('_', expand=True).set_names(['REGIÃO', 'ESTADO', 'CODIGO', 'NOME'])
df = df.reorder_levels(['REGIÃO', 'ESTADO', 'NOME', 'CODIGO'])
df.to_csv(f'datasets/%_dias_off.csv', sep=';')
df

Data                                         2020
REGIÃO ESTADO NOME               CODIGO          
CO     DF     BRASILIA           A001    0.000000
              BRAZLANDIA         A042    0.000000
              AGUAS EMENDADAS    A045    0.000000
              GAMA (PONTE ALTA)  A046    0.000000
              PARANOA (COOPA-DF) A047    0.000000
...                                           ...
S      SC     ARARANGUA          A867    0.000000
              ITAJAI             A868    0.169399
              RANCHO QUEIMADO    A870    0.360656
              CHAPECO            A895    0.000000
              CAMPOS NOVOS       A898    0.000000

[589 rows x 1 columns]

In [16]:
path = 'datasets/agregados/'
todos = []
for file in listdir(path):
    data = pd.read_csv(path+file, sep=';', index_col = [0], nrows=3).iloc[:, -1]
    loc = file[:-4].split('_')
    data['REGIÃO'] = loc[0]
    data['ESTADO'] = loc[1]
    data['NOME'] = loc[3]
    data['CODIGO'] = loc[2]
    #data = data.rename(loc[2])
    todos.append(data)
coords = pd.concat(todos, axis=1).T.set_index(['REGIÃO', 'ESTADO', 'NOME', 'CODIGO'])
coords

LATITUDE  LONGITUDE ALTITUDE
REGIÃO ESTADO NOME                 CODIGO                               
NE     CE     BARBALHA             A315    -7.300833 -39.271111   409.41
N      PA     TOME ACU             A213      -2.5925 -48.360556    42.95
CO     MS     BRASILANDIA          S705   -21.298278 -52.068917    345.0
N      AC     MARECHAL THAUMATURGO A137        -8.95 -72.786667   220.98
NE     CE     ACARAU               A360    -3.121067 -40.087288    67.15
...                                              ...        ...      ...
       PI     PAULISTANA           A330    -8.132288 -41.142945    376.0
SE     MG     AIMORES              A534   -19.532768 -41.090801   287.74
              DIAMANTINA           A537   -18.231052 -43.648269  1359.25
S      SC     FLORIANOPOLIS        A806    -27.60253 -48.620096     4.87
NE     BA     BRUMADO              A433   -14.181886 -41.672281   473.37

[589 rows x 3 columns]

In [17]:
funcs = {
    'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)': lambda x: x.sum(min_count=1),
    'RADIACAO GLOBAL (KJ/m²)': 'mean',
    'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)': 'max' ,
    'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)': 'min' ,
    'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)': 'max' ,
    'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)': 'min' ,
    'VENTO, VELOCIDADE HORARIA (m/s)': 'mean'
}

i = 0
for file in listdir(path_output):
    print(file)
    path_in = f'{path_output}/{file}'
    path_out = f'datasets/diarios/{file}'
    
    pd.read_csv(path_in, sep=';', nrows=3, index_col=[0]) \
            .to_csv(path_out, sep=';')
    
    pd.read_csv( path_in, sep=';', index_col=[0, 1], skiprows=4) \
            .groupby(level=0).apply(lambda group: group.agg(funcs)) \
            .to_csv(path_out, sep=';', mode='a')

NE_CE_A315_BARBALHA.csv
N_PA_A213_TOME ACU.csv
CO_MS_S705_BRASILANDIA.csv
N_AC_A137_MARECHAL THAUMATURGO.csv
NE_CE_A360_ACARAU.csv
NE_AL_A371_PIRANHAS.csv
NE_RN_A340_APODI.csv
CO_DF_A042_BRAZLANDIA.csv
N_RO_A938_VILHENA.csv
NE_MA_A225_IMPERATRIZ.csv
CO_MT_A902_TANGARA DA SERRA.csv
SE_SP_A727_LINS.csv
S_PR_A824_ICARAIMA.csv
SE_ES_A631_ECOPORANGA.csv
S_RS_A804_SANTANA DO LIVRAMENTO.csv
N_TO_A049_COLINAS DO TOCANTINS.csv
NE_PB_A310_AREIA.csv
S_SC_A814_URUSSANGA.csv
S_RS_A879_CANELA.csv
N_AM_A110_BOCA DO ACRE.csv
NE_BA_A442_EUCLIDES DA CUNHA.csv
NE_MA_A206_CHAPADINHA.csv
SE_MG_A547_SAO ROMAO.csv
SE_SP_A705_BAURU.csv
CO_MS_A719_AQUIDAUANA.csv
S_SC_A859_CACADOR.csv
CO_GO_A023_CAIAPONIA.csv
SE_ES_A657_AFONSO CLAUDIO.csv
CO_MT_A924_ALTA FLORESTA.csv
S_RS_A839_PASSO FUNDO.csv
CO_GO_A017_POSSE.csv
CO_MT_A908_AGUA BOA.csv
SE_MG_F501_BELO HORIZONTE - CERCADINHO.csv
SE_MG_A567_MACHADO.csv
SE_SP_A771_SAO PAULO - INTERLAGOS.csv
N_TO_A041_MARIANOPOLIS DO TO.csv
NE_BA_A414_VITORIA DA CONQUISTA.csv
SE_E

SE_SP_A746_BARRA DO TURVO.csv
N_AM_A111_LABREA.csv
S_SC_A860_CURITIBANOS.csv
S_SC_A866_Laguna  - Farol de Santa Marta.csv
NE_BA_A435_UAUA.csv
SE_RJ_A626_RIO CLARO.csv
CO_DF_A046_GAMA (PONTE ALTA).csv
N_TO_A044_ARAGUATINS.csv
SE_RJ_A627_NITEROI.csv
SE_SP_A711_SAO CARLOS.csv
NE_BA_A446_ITAPETINGA.csv
CO_MS_S713_NOVA ANDRADINA.csv
CO_GO_A024_ALTO PARAISO DE GOIAS.csv
CO_GO_A027_PARAUNA.csv
CO_MS_A709_IVINHEMA.csv
N_PA_A253_ALTAMIRA.csv
NE_BA_A436_QUEIMADAS.csv
SE_SP_A714_ITAPEVA.csv
CO_MS_A731_MARACAJU.csv
NE_PB_A333_SAO GONCALO.csv
SE_RJ_A608_MACAE.csv
CO_MS_A722_MIRANDA.csv
N_PA_A202_CASTANHAL.csv
N_TO_A055_LAGOA DA CONFUSAO.csv
S_PR_A874_SAO MATEUS DO SUL.csv
CO_MT_A937_PONTES E LACERDA.csv
NE_BA_A432_BURITIRAMA.csv
SE_SP_A734_VALPARAISO.csv
N_TO_A043_CAMPOS LINDOS.csv
N_PA_A201_BELEM.csv
S_PR_A876_CLEVELANDIA.csv
N_AM_A124_URUCARA.csv
SE_MG_A556_MANHUACU.csv
SE_MG_A571_PARACATU.csv
SE_SP_A747_PRADOPOLIS.csv
SE_SP_A766_REGISTRO.csv
SE_MG_A518_JUIZ DE FORA.csv
CO_MS_A760_COSTA RICA.csv


In [18]:
#unir arquivos por região
path = 'datasets/diarios/'
todos = []
for file in listdir(path):
    path_file = f'{path}{file}'
    regiao, estado, codigo, nome = file[:-4].split('_')

    loc = pd.read_csv(path_file, sep=';', nrows=3, index_col=[0])
    df = pd.read_csv( path_file, sep=';', index_col=[0], skiprows=4)
    
    for ano in loc:
        indexes = df.index.str.startswith(ano)
        for j in ['LATITUDE', 'LONGITUDE', 'ALTITUDE']:
            df.loc[indexes, j] = loc[ano][j]
        
    df['REGIAO'] = regiao
    df['ESTADO'] = estado
    df['CODIGO'] = codigo
    df['NOME'] = nome

    todos.append( df.set_index(['REGIAO', 'ESTADO', 'CODIGO', 'NOME', 'LATITUDE', 'LONGITUDE', 'ALTITUDE'], append=True) )
pd.concat(todos).to_csv('datasets/consolidado.csv', sep=';')